In [1]:
import tensorflow as tf;
import os;
os.environ['KMP_DUPLICATE_LIB_OK']='True'; # for mac

In [57]:
#gamma divergence
def gamma_divergence(y_true, y_pred):
    P = y_true;
    Q = y_pred;
    n = tf.shape(P)[0];
    k = tf.shape(P)[1];
    gamma = tf.constant(_gamma); # gamma = 0.5
    epsilon = tf.math.add(tf.constant(1.0), gamma);
    g = tf.ones([n,k],tf.float32) * gamma;
    e = tf.ones([n,k],tf.float32) * epsilon;
    Qg = tf.math.pow(Q,g);
    Qe = tf.math.pow(Q,e);
    Pe  = tf.math.pow(P,e);
    PQg = tf.math.multiply(P,Qg);
    sumPQg = tf.reduce_sum(PQg,axis=1);
    sumQe = tf.reduce_sum(Qe,axis=1);
    sumPe = tf.reduce_sum(Pe,axis=1);
    lnsumPe = tf.math.log(sumPe);
    lnsumPQg = tf.math.log(sumPQg);
    lnsumQe = tf.math.log(sumQe);
    return tf.reduce_mean(
        lnsumPe + gamma * lnsumQe - epsilon * lnsumPQg
    ) / (gamma * epsilon);

In [58]:
# kl divergence
def kl_divergence(y_true, y_pred):
    n = tf.shape(y_true)[0];
    k = tf.shape(y_true)[1];
    P = y_true + tf.ones([n,k]) * 10**-6;
    Q= y_pred;
    lnP = tf.math.log(P);
    lnQ = tf.math.log(Q);
    PlnP = tf.math.multiply(P,lnP);
    PlnQ = tf.math.multiply(P,lnQ);
    return tf.reduce_mean(
        tf.math.reduce_sum(PlnP-PlnQ,axis=1)
    );

In [59]:
batch_size = 100;
num_classes = 10;
epochs = 10;
# prepare data
# mnist
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data();
# fashon mnist
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data();


img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')   # int型をfloat32型に変換
x_test = x_test.astype('float32')
x_train /= 255                        # [0-255]の値を[0.0-1.0]に変換
x_test /= 255
print(x_train.shape[0], 'train samples');
print(x_test.shape[0], 'test samples');
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [60]:
#### My Model  with Projection Layer ####
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=1, padding='same', activation='relu',input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2));
model.add(tf.keras.layers.Conv2D(16, kernel_size=(2,2),strides=1, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2));
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
# gamma-divergence
model.compile(loss=gamma_divergence, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy']);
# KL-divergence
#model.compile(loss=tf.keras.losses.kullback_leibler_divergence, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy']);
#model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
# start training
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 18s 296us/sample - loss: 0.2337 - accuracy: 0.8357 - val_loss: 0.1764 - val_accuracy: 0.8751
Epoch 2/10
60000/60000 [==============================] - 19s 309us/sample - loss: 0.1591 - accuracy: 0.8868 - val_loss: 0.1645 - val_accuracy: 0.8780
Epoch 3/10
60000/60000 [==============================] - 19s 312us/sample - loss: 0.1395 - accuracy: 0.8992 - val_loss: 0.1468 - val_accuracy: 0.8922
Epoch 4/10
60000/60000 [==============================] - 18s 297us/sample - loss: 0.1261 - accuracy: 0.9076 - val_loss: 0.1583 - val_accuracy: 0.8889
Epoch 5/10
60000/60000 [==============================] - 18s 302us/sample - loss: 0.1165 - accuracy: 0.9137 - val_loss: 0.1463 - val_accuracy: 0.8962
Epoch 6/10
60000/60000 [==============================] - 19s 324us/sample - loss: 0.1084 - accuracy: 0.9195 - val_loss: 0.1361 - val_accuracy: 0.8983
Epoch 7/10
60000/60000 [====================

In [ ]:
model.summary();